In [ ]:
#1. modified all(long description won't change)

In [1]:
from collections import defaultdict
from collections import Counter
import MySQLdb
import pandas as pd
import numpy as np
import sys
reload(sys)
sys.setdefaultencoding('utf8')
#import numexpr as ne
#import matplotlib.pyplot as plt
#%matplotlib inline
conn = MySQLdb.connect(
                     host='localhost', port=3306, user='***', passwd='***', db = 'eclipse2011',
                     charset='utf8')

In [2]:
bug_info = pd.read_sql("SELECT bug_id, short_desc, product_id, component_id, priority FROM bugs WHERE bug_id >= 214049 and bug_id <= 259814;", conn)
bug_info.set_index('bug_id', inplace=True)
bug_info.shape

(45764, 4)

In [3]:
pd.read_sql('SELECT id, name, description from fielddefs where name in ("short_desc", "product", "component", "priority")', conn)

,id,name,description
0,14,component,Component
1,13,priority,Priority
2,3,product,Product
3,2,short_desc,Summary


In [4]:
bug_info.to_pickle("original_info.pkl")

In [5]:
df = pd.read_pickle('original_info.pkl').reset_index()

from nltk.tokenize import WordPunctTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
tokenizer = WordPunctTokenizer()
stemmer = PorterStemmer()

def get_bigrams(string):
    try:
        l = filter(lambda x: x.isalpha(), tokenizer.tokenize(string.lower()))
        bigrams = []
        for i in range(1, len(l)):
            bigrams.append(l[i-1] + ' ' + l[i])
        return bigrams
    except Exception:
        return []

def get_trigrams(string):
    try:
        l = filter(lambda x: x.isalpha(), tokenizer.tokenize(string.lower()))
        bigrams = []
        for i in range(2, len(l)):
            bigrams.append(l[i-2] + ' ' + l[i-1] + ' ' + l[i])
        return bigrams
    except Exception:
        return []

short_desc_bag = df['short_desc'].apply(lambda string: Counter(filter(lambda x: x not in stopwords and x.isalpha(), map(lambda x: stemmer.stem(x), tokenizer.tokenize(string.lower())))))
short_desc_bag.to_pickle('short_desc_bag.pkl')
short_desc_bag = None

short_desc_bi_bag = df['short_desc'].apply(lambda string: Counter(get_bigrams(string)))
short_desc_bi_bag.to_pickle('short_desc_bi_bag.pkl')
short_desc_bi_bag = None

short_desc_tri_bag = df['short_desc'].apply(lambda string: Counter(get_trigrams(string)))
short_desc_tri_bag.to_pickle('short_desc_tri_bag.pkl')
short_desc_tri_bag = None

df.to_pickle('bug_info.pkl')
df = None

description = pd.read_sql('select longdescs.bug_id, longdescs.thetext from longdescs, (select bug_id, MIN(comment_id) as comment_id from longdescs group by bug_id) as t where longdescs.bug_id = t.bug_id and longdescs.comment_id = t.comment_id and longdescs.bug_id >= 214049 and longdescs.bug_id <= 259814', conn)

description_bag = description['thetext'].apply(lambda string: Counter(filter(lambda x: x not in stopwords and x.isalpha(), map(lambda x: stemmer.stem(x), tokenizer.tokenize(string.lower())))))
description_bag.to_pickle('description_bag.pkl')
description_bag = None

description_bi_bag = description['thetext'].apply(lambda string: Counter(get_bigrams(string)))
description_bi_bag.to_pickle('description_bi_bag.pkl')
description_bi_bag = None

description_tri_bag = description['thetext'].apply(lambda string: Counter(get_trigrams(string)))
description_tri_bag.to_pickle('description_tri_bag.pkl')
description_tri_bag = None

description.to_pickle('description.pkl')
description = None

bags = [None] * 6
bags[0] = 'short_desc_bag.pkl'
bags[1] = 'short_desc_bi_bag.pkl'
bags[2] = 'short_desc_tri_bag.pkl'
bags[3] = 'description_bag.pkl'
bags[4] = 'description_bi_bag.pkl'
bags[5] = 'description_tri_bag.pkl'

all_words = set()
for filename in bags:
    series = pd.read_pickle(filename)
    for data in series:
        for word in data:
            all_words.add(word)
            
dictionary = dict((v,k) for k, v in enumerate(all_words))

all_words = None

import pickle
f = open('dictionary.pkl', 'wb')
pickle.dump(dictionary, f)
f.close()

import pickle
import pandas as pd
f = open('dictionary.pkl', 'rb')
dictionary = pickle.load(f)

df = pd.read_pickle('bug_info.pkl')
bags = [None] * 6
bags[0] = 'short_desc_bag.pkl'
bags[1] = 'short_desc_bi_bag.pkl'
bags[2] = 'short_desc_tri_bag.pkl'
bags[3] = 'description_bag.pkl'
bags[4] = 'description_bi_bag.pkl'
bags[5] = 'description_tri_bag.pkl'

for filename in bags:
    series = pd.read_pickle(filename)
    df[filename[:-4]] = series

modified = df['priority'].replace('--', 6).replace('P1', 1).replace('P2', 2).replace('P3', 3).replace('P4', 4).replace('P5', 5)
df['priority'] = modified
df.to_pickle('all_in_one.pkl')

In [6]:
import pickle
import pandas as pd
f = open('dictionary.pkl', 'rb')
dictionary = pickle.load(f)
df = pd.read_pickle('all_in_one.pkl')
df.set_index('bug_id', inplace=True)
lines = open('large-eclipse.txt').readlines()

#ID   0
#S-U  1
#S-B  2
#S-T  3
#D-U  4
#D-B  5
#D-T  6
#A-U  7
#A-B  8
#A-T  9
#DID  10
#VERSION       11
#COMPONENT     12
#SUB_COMPONENT 13
#TYPE          14
#PRIORITY      15
with open('eclipse_modified_all.txt','w') as out:
    for i in range(0, len(lines), 16):
        ID = int(lines[i].strip().split('=')[1])
        id_line = lines[i].strip()
        did_line = lines[i + 10].strip()
        version_line = lines[i + 11].strip()
        type_line = lines[i + 14].strip()
        if ID in df.index:
            out.write(id_line)
            out.write('\n')
            out.write( 'S-U=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bag'].items())))
            out.write('\n')
            out.write( 'S-B=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bi_bag'].items())))
            out.write('\n')
            out.write( 'S-T=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_tri_bag'].items())))
            out.write('\n')
            out.write( 'D-U=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'description_bag'].items())))
            out.write('\n')
            out.write( 'D-B=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'description_bi_bag'].items())))
            out.write('\n')
            out.write( 'D-T=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'description_tri_bag'].items())))
            out.write('\n')
            out.write( 'A-U=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bag'].items() + df.ix[ID, 'description_bag'].items())))
            out.write('\n')
            out.write( 'A-B=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bag'].items() + df.ix[ID, 'description_bi_bag'].items())))
            out.write('\n')
            out.write( 'A-T=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bag'].items() + df.ix[ID, 'description_tri_bag'].items())))
            out.write('\n')
            out.write( did_line)
            out.write('\n')
            out.write( version_line)
            out.write('\n')
            out.write( "COMPONENT=" + str(df.ix[ID, 'product_id']))
            out.write('\n')
            out.write( "SUB-COMPONENT=" + str(df.ix[ID, 'component_id']))
            out.write('\n')
            out.write( type_line)
            out.write('\n')
            out.write( "PRIORITY=" + str(df.ix[ID, 'priority']))
            out.write('\n')

In [ ]:
#2. initial summary

In [1]:
from collections import defaultdict
from collections import Counter
import MySQLdb
import pandas as pd
import numpy as np
import sys
reload(sys)
sys.setdefaultencoding('utf8')

conn = MySQLdb.connect(
                     host='localhost', port=3306, user='***', passwd='***', db = 'eclipse2011',
                     charset='utf8')

bug_info = pd.read_sql("SELECT bug_id, short_desc, product_id, component_id, priority FROM bugs WHERE bug_id >= 214049 and bug_id <= 259814;", conn)
bug_info.set_index('bug_id', inplace=True)
activities = pd.read_sql('SELECT * FROM bugs_activity where bug_id >= 214049 and bug_id <= 259814 and fieldid = 2 ' , conn)

product_df = pd.read_sql('SELECT  name,id FROM products ', conn)
product_df.set_index('name', inplace=True)
product_mapping = product_df['id'].to_dict()

component_df = pd.read_sql('SELECT  name,id FROM components ', conn)
component_df.set_index('name', inplace=True)
component_mapping = component_df['id'].to_dict()

activities.sort('bug_when', ascending=False, inplace=True)

after = bug_info.copy()

for rownum in range(0, activities.shape[0]):
    row = activities.iloc[rownum]
    bug_id = row['bug_id']
    field = row['fieldid']
    removed = row['removed']
    try:
        if field == 2:
            after.ix[bug_id, 'short_desc'] = removed
    except Exception:
        pass

df = after.copy().reset_index()

from nltk.tokenize import WordPunctTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
tokenizer = WordPunctTokenizer()
stemmer = PorterStemmer()

def get_bigrams(string):
    try:
        l = filter(lambda x: x.isalpha(), tokenizer.tokenize(string.lower()))
        bigrams = []
        for i in range(1, len(l)):
            bigrams.append(l[i-1] + ' ' + l[i])
        return bigrams
    except Exception:
        return []

def get_trigrams(string):
    try:
        l = filter(lambda x: x.isalpha(), tokenizer.tokenize(string.lower()))
        bigrams = []
        for i in range(2, len(l)):
            bigrams.append(l[i-2] + ' ' + l[i-1] + ' ' + l[i])
        return bigrams
    except Exception:
        return []

short_desc_bag = df['short_desc'].apply(lambda string: Counter(filter(lambda x: x not in stopwords and x.isalpha(), map(lambda x: stemmer.stem(x), tokenizer.tokenize(string.lower())))))
short_desc_bag.to_pickle('short_desc_bag.pkl')
short_desc_bag = None

short_desc_bi_bag = df['short_desc'].apply(lambda string: Counter(get_bigrams(string)))
short_desc_bi_bag.to_pickle('short_desc_bi_bag.pkl')
short_desc_bi_bag = None

short_desc_tri_bag = df['short_desc'].apply(lambda string: Counter(get_trigrams(string)))
short_desc_tri_bag.to_pickle('short_desc_tri_bag.pkl')
short_desc_tri_bag = None

df.to_pickle('bug_info.pkl')
df = None

description = pd.read_sql('select longdescs.bug_id, longdescs.thetext from longdescs, (select bug_id, MIN(comment_id) as comment_id from longdescs group by bug_id) as t where longdescs.bug_id = t.bug_id and longdescs.comment_id = t.comment_id and longdescs.bug_id >= 214049 and longdescs.bug_id <= 259814', conn)

description_bag = description['thetext'].apply(lambda string: Counter(filter(lambda x: x not in stopwords and x.isalpha(), map(lambda x: stemmer.stem(x), tokenizer.tokenize(string.lower())))))
description_bag.to_pickle('description_bag.pkl')
description_bag = None

description_bi_bag = description['thetext'].apply(lambda string: Counter(get_bigrams(string)))
description_bi_bag.to_pickle('description_bi_bag.pkl')
description_bi_bag = None

description_tri_bag = description['thetext'].apply(lambda string: Counter(get_trigrams(string)))
description_tri_bag.to_pickle('description_tri_bag.pkl')
description_tri_bag = None

description.to_pickle('description.pkl')
description = None

bags = [None] * 6
bags[0] = 'short_desc_bag.pkl'
bags[1] = 'short_desc_bi_bag.pkl'
bags[2] = 'short_desc_tri_bag.pkl'
bags[3] = 'description_bag.pkl'
bags[4] = 'description_bi_bag.pkl'
bags[5] = 'description_tri_bag.pkl'

all_words = set()
for filename in bags:
    series = pd.read_pickle(filename)
    for data in series:
        for word in data:
            all_words.add(word)
            
dictionary = dict((v,k) for k, v in enumerate(all_words))

all_words = None

import pickle
f = open('dictionary.pkl', 'wb')
pickle.dump(dictionary, f)
f.close()

import pickle
import pandas as pd
f = open('dictionary.pkl', 'rb')
dictionary = pickle.load(f)

df = pd.read_pickle('bug_info.pkl')
bags = [None] * 6
bags[0] = 'short_desc_bag.pkl'
bags[1] = 'short_desc_bi_bag.pkl'
bags[2] = 'short_desc_tri_bag.pkl'
bags[3] = 'description_bag.pkl'
bags[4] = 'description_bi_bag.pkl'
bags[5] = 'description_tri_bag.pkl'

for filename in bags:
    series = pd.read_pickle(filename)
    df[filename[:-4]] = series

modified = df['priority'].replace('--', 6).replace('P1', 1).replace('P2', 2).replace('P3', 3).replace('P4', 4).replace('P5', 5)
df['priority'] = modified

df.to_pickle('all_in_one.pkl')

In [2]:
import pickle
import pandas as pd
f = open('dictionary.pkl', 'rb')
dictionary = pickle.load(f)
df = pd.read_pickle('all_in_one.pkl')
df.set_index('bug_id', inplace=True)
lines = open('large-eclipse.txt').readlines()

#ID   0
#S-U  1
#S-B  2
#S-T  3
#D-U  4
#D-B  5
#D-T  6
#A-U  7
#A-B  8
#A-T  9
#DID  10
#VERSION       11
#COMPONENT     12
#SUB_COMPONENT 13
#TYPE          14
#PRIORITY      15
with open('eclipse_initial_summary.txt','w') as out:
    for i in range(0, len(lines), 16):
        ID = int(lines[i].strip().split('=')[1])
        id_line = lines[i].strip()
        did_line = lines[i + 10].strip()
        version_line = lines[i + 11].strip()
        type_line = lines[i + 14].strip()
        if ID in df.index:
            out.write(id_line)
            out.write('\n')
            out.write( 'S-U=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bag'].items())))
            out.write('\n')
            out.write( 'S-B=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bi_bag'].items())))
            out.write('\n')
            out.write( 'S-T=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_tri_bag'].items())))
            out.write('\n')
            out.write( 'D-U=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'description_bag'].items())))
            out.write('\n')
            out.write( 'D-B=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'description_bi_bag'].items())))
            out.write('\n')
            out.write( 'D-T=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'description_tri_bag'].items())))
            out.write('\n')
            out.write( 'A-U=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bag'].items() + df.ix[ID, 'description_bag'].items())))
            out.write('\n')
            out.write( 'A-B=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bag'].items() + df.ix[ID, 'description_bi_bag'].items())))
            out.write('\n')
            out.write( 'A-T=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bag'].items() + df.ix[ID, 'description_tri_bag'].items())))
            out.write('\n')
            out.write( did_line)
            out.write('\n')
            out.write( version_line)
            out.write('\n')
            out.write( "COMPONENT=" + str(df.ix[ID, 'product_id']))
            out.write('\n')
            out.write( "SUB-COMPONENT=" + str(df.ix[ID, 'component_id']))
            out.write('\n')
            out.write( type_line)
            out.write('\n')
            out.write( "PRIORITY=" + str(df.ix[ID, 'priority']))
            out.write('\n')

In [ ]:
#3. initial product 

In [1]:
from collections import defaultdict
from collections import Counter
import MySQLdb
import pandas as pd
import numpy as np
import sys
reload(sys)
sys.setdefaultencoding('utf8')

conn = MySQLdb.connect(
                     host='localhost', port=3306, user='***', passwd='***', db = 'eclipse2011',
                     charset='utf8')

bug_info = pd.read_sql("SELECT bug_id, short_desc, product_id, component_id, priority FROM bugs WHERE bug_id >= 214049 and bug_id <= 259814;", conn)
bug_info.set_index('bug_id', inplace=True)
activities = pd.read_sql('SELECT * FROM bugs_activity where bug_id >= 214049 and bug_id <= 259814 and fieldid = 3 ' , conn)

product_df = pd.read_sql('SELECT  name,id FROM products ', conn)
product_df.set_index('name', inplace=True)
product_mapping = product_df['id'].to_dict()

component_df = pd.read_sql('SELECT  name,id FROM components ', conn)
component_df.set_index('name', inplace=True)
component_mapping = component_df['id'].to_dict()

activities.sort('bug_when', ascending=False, inplace=True)

after = bug_info.copy()

for rownum in range(0, activities.shape[0]):
    row = activities.iloc[rownum]
    bug_id = row['bug_id']
    field = row['fieldid']
    removed = row['removed']
    try:
        if field == 3:
            after.ix[bug_id, 'product_id'] = product_mapping[removed]
    except Exception:
        pass

df = after.copy().reset_index()

from nltk.tokenize import WordPunctTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
tokenizer = WordPunctTokenizer()
stemmer = PorterStemmer()

def get_bigrams(string):
    try:
        l = filter(lambda x: x.isalpha(), tokenizer.tokenize(string.lower()))
        bigrams = []
        for i in range(1, len(l)):
            bigrams.append(l[i-1] + ' ' + l[i])
        return bigrams
    except Exception:
        return []

def get_trigrams(string):
    try:
        l = filter(lambda x: x.isalpha(), tokenizer.tokenize(string.lower()))
        bigrams = []
        for i in range(2, len(l)):
            bigrams.append(l[i-2] + ' ' + l[i-1] + ' ' + l[i])
        return bigrams
    except Exception:
        return []

short_desc_bag = df['short_desc'].apply(lambda string: Counter(filter(lambda x: x not in stopwords and x.isalpha(), map(lambda x: stemmer.stem(x), tokenizer.tokenize(string.lower())))))
short_desc_bag.to_pickle('short_desc_bag.pkl')
short_desc_bag = None

short_desc_bi_bag = df['short_desc'].apply(lambda string: Counter(get_bigrams(string)))
short_desc_bi_bag.to_pickle('short_desc_bi_bag.pkl')
short_desc_bi_bag = None

short_desc_tri_bag = df['short_desc'].apply(lambda string: Counter(get_trigrams(string)))
short_desc_tri_bag.to_pickle('short_desc_tri_bag.pkl')
short_desc_tri_bag = None

df.to_pickle('bug_info.pkl')
df = None

description = pd.read_sql('select longdescs.bug_id, longdescs.thetext from longdescs, (select bug_id, MIN(comment_id) as comment_id from longdescs group by bug_id) as t where longdescs.bug_id = t.bug_id and longdescs.comment_id = t.comment_id and longdescs.bug_id >= 214049 and longdescs.bug_id <= 259814', conn)

description_bag = description['thetext'].apply(lambda string: Counter(filter(lambda x: x not in stopwords and x.isalpha(), map(lambda x: stemmer.stem(x), tokenizer.tokenize(string.lower())))))
description_bag.to_pickle('description_bag.pkl')
description_bag = None

description_bi_bag = description['thetext'].apply(lambda string: Counter(get_bigrams(string)))
description_bi_bag.to_pickle('description_bi_bag.pkl')
description_bi_bag = None

description_tri_bag = description['thetext'].apply(lambda string: Counter(get_trigrams(string)))
description_tri_bag.to_pickle('description_tri_bag.pkl')
description_tri_bag = None

description.to_pickle('description.pkl')
description = None

bags = [None] * 6
bags[0] = 'short_desc_bag.pkl'
bags[1] = 'short_desc_bi_bag.pkl'
bags[2] = 'short_desc_tri_bag.pkl'
bags[3] = 'description_bag.pkl'
bags[4] = 'description_bi_bag.pkl'
bags[5] = 'description_tri_bag.pkl'

all_words = set()
for filename in bags:
    series = pd.read_pickle(filename)
    for data in series:
        for word in data:
            all_words.add(word)
            
dictionary = dict((v,k) for k, v in enumerate(all_words))

all_words = None

import pickle
f = open('dictionary.pkl', 'wb')
pickle.dump(dictionary, f)
f.close()

import pickle
import pandas as pd
f = open('dictionary.pkl', 'rb')
dictionary = pickle.load(f)

df = pd.read_pickle('bug_info.pkl')
bags = [None] * 6
bags[0] = 'short_desc_bag.pkl'
bags[1] = 'short_desc_bi_bag.pkl'
bags[2] = 'short_desc_tri_bag.pkl'
bags[3] = 'description_bag.pkl'
bags[4] = 'description_bi_bag.pkl'
bags[5] = 'description_tri_bag.pkl'

for filename in bags:
    series = pd.read_pickle(filename)
    df[filename[:-4]] = series

modified = df['priority'].replace('--', 6).replace('P1', 1).replace('P2', 2).replace('P3', 3).replace('P4', 4).replace('P5', 5)
df['priority'] = modified

df.to_pickle('all_in_one.pkl')

In [2]:
import pickle
import pandas as pd
f = open('dictionary.pkl', 'rb')
dictionary = pickle.load(f)
df = pd.read_pickle('all_in_one.pkl')
df.set_index('bug_id', inplace=True)
lines = open('large-eclipse.txt').readlines()

#ID   0
#S-U  1
#S-B  2
#S-T  3
#D-U  4
#D-B  5
#D-T  6
#A-U  7
#A-B  8
#A-T  9
#DID  10
#VERSION       11
#COMPONENT     12
#SUB_COMPONENT 13
#TYPE          14
#PRIORITY      15
with open('eclipse_initial_product.txt','w') as out:
    for i in range(0, len(lines), 16):
        ID = int(lines[i].strip().split('=')[1])
        id_line = lines[i].strip()
        did_line = lines[i + 10].strip()
        version_line = lines[i + 11].strip()
        type_line = lines[i + 14].strip()
        if ID in df.index:
            out.write(id_line)
            out.write('\n')
            out.write( 'S-U=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bag'].items())))
            out.write('\n')
            out.write( 'S-B=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bi_bag'].items())))
            out.write('\n')
            out.write( 'S-T=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_tri_bag'].items())))
            out.write('\n')
            out.write( 'D-U=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'description_bag'].items())))
            out.write('\n')
            out.write( 'D-B=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'description_bi_bag'].items())))
            out.write('\n')
            out.write( 'D-T=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'description_tri_bag'].items())))
            out.write('\n')
            out.write( 'A-U=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bag'].items() + df.ix[ID, 'description_bag'].items())))
            out.write('\n')
            out.write( 'A-B=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bag'].items() + df.ix[ID, 'description_bi_bag'].items())))
            out.write('\n')
            out.write( 'A-T=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bag'].items() + df.ix[ID, 'description_tri_bag'].items())))
            out.write('\n')
            out.write( did_line)
            out.write('\n')
            out.write( version_line)
            out.write('\n')
            out.write( "COMPONENT=" + str(df.ix[ID, 'product_id']))
            out.write('\n')
            out.write( "SUB-COMPONENT=" + str(df.ix[ID, 'component_id']))
            out.write('\n')
            out.write( type_line)
            out.write('\n')
            out.write( "PRIORITY=" + str(df.ix[ID, 'priority']))
            out.write('\n')

In [ ]:
#4. initial component

In [1]:
from collections import defaultdict
from collections import Counter
import MySQLdb
import pandas as pd
import numpy as np
import sys
reload(sys)
sys.setdefaultencoding('utf8')

conn = MySQLdb.connect(
                     host='localhost', port=3306, user='***', passwd='***', db = 'eclipse2011',
                     charset='utf8')

bug_info = pd.read_sql("SELECT bug_id, short_desc, product_id, component_id, priority FROM bugs WHERE bug_id >= 214049 and bug_id <= 259814;", conn)
bug_info.set_index('bug_id', inplace=True)
activities = pd.read_sql('SELECT * FROM bugs_activity where bug_id >= 214049 and bug_id <= 259814 and fieldid = 14 ' , conn)

product_df = pd.read_sql('SELECT  name,id FROM products ', conn)
product_df.set_index('name', inplace=True)
product_mapping = product_df['id'].to_dict()

component_df = pd.read_sql('SELECT  name,id FROM components ', conn)
component_df.set_index('name', inplace=True)
component_mapping = component_df['id'].to_dict()

activities.sort('bug_when', ascending=False, inplace=True)

after = bug_info.copy()

for rownum in range(0, activities.shape[0]):
    row = activities.iloc[rownum]
    bug_id = row['bug_id']
    field = row['fieldid']
    removed = row['removed']
    try:
        if field == 14:
            after.ix[bug_id, 'component_id'] = component_mapping[removed]
    except Exception:
        pass

df = after.copy().reset_index()

from nltk.tokenize import WordPunctTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
tokenizer = WordPunctTokenizer()
stemmer = PorterStemmer()

def get_bigrams(string):
    try:
        l = filter(lambda x: x.isalpha(), tokenizer.tokenize(string.lower()))
        bigrams = []
        for i in range(1, len(l)):
            bigrams.append(l[i-1] + ' ' + l[i])
        return bigrams
    except Exception:
        return []

def get_trigrams(string):
    try:
        l = filter(lambda x: x.isalpha(), tokenizer.tokenize(string.lower()))
        bigrams = []
        for i in range(2, len(l)):
            bigrams.append(l[i-2] + ' ' + l[i-1] + ' ' + l[i])
        return bigrams
    except Exception:
        return []

short_desc_bag = df['short_desc'].apply(lambda string: Counter(filter(lambda x: x not in stopwords and x.isalpha(), map(lambda x: stemmer.stem(x), tokenizer.tokenize(string.lower())))))
short_desc_bag.to_pickle('short_desc_bag.pkl')
short_desc_bag = None

short_desc_bi_bag = df['short_desc'].apply(lambda string: Counter(get_bigrams(string)))
short_desc_bi_bag.to_pickle('short_desc_bi_bag.pkl')
short_desc_bi_bag = None

short_desc_tri_bag = df['short_desc'].apply(lambda string: Counter(get_trigrams(string)))
short_desc_tri_bag.to_pickle('short_desc_tri_bag.pkl')
short_desc_tri_bag = None

df.to_pickle('bug_info.pkl')
df = None

description = pd.read_sql('select longdescs.bug_id, longdescs.thetext from longdescs, (select bug_id, MIN(comment_id) as comment_id from longdescs group by bug_id) as t where longdescs.bug_id = t.bug_id and longdescs.comment_id = t.comment_id and longdescs.bug_id >= 214049 and longdescs.bug_id <= 259814', conn)

description_bag = description['thetext'].apply(lambda string: Counter(filter(lambda x: x not in stopwords and x.isalpha(), map(lambda x: stemmer.stem(x), tokenizer.tokenize(string.lower())))))
description_bag.to_pickle('description_bag.pkl')
description_bag = None

description_bi_bag = description['thetext'].apply(lambda string: Counter(get_bigrams(string)))
description_bi_bag.to_pickle('description_bi_bag.pkl')
description_bi_bag = None

description_tri_bag = description['thetext'].apply(lambda string: Counter(get_trigrams(string)))
description_tri_bag.to_pickle('description_tri_bag.pkl')
description_tri_bag = None

description.to_pickle('description.pkl')
description = None

bags = [None] * 6
bags[0] = 'short_desc_bag.pkl'
bags[1] = 'short_desc_bi_bag.pkl'
bags[2] = 'short_desc_tri_bag.pkl'
bags[3] = 'description_bag.pkl'
bags[4] = 'description_bi_bag.pkl'
bags[5] = 'description_tri_bag.pkl'

all_words = set()
for filename in bags:
    series = pd.read_pickle(filename)
    for data in series:
        for word in data:
            all_words.add(word)
            
dictionary = dict((v,k) for k, v in enumerate(all_words))

all_words = None

import pickle
f = open('dictionary.pkl', 'wb')
pickle.dump(dictionary, f)
f.close()

import pickle
import pandas as pd
f = open('dictionary.pkl', 'rb')
dictionary = pickle.load(f)

df = pd.read_pickle('bug_info.pkl')
bags = [None] * 6
bags[0] = 'short_desc_bag.pkl'
bags[1] = 'short_desc_bi_bag.pkl'
bags[2] = 'short_desc_tri_bag.pkl'
bags[3] = 'description_bag.pkl'
bags[4] = 'description_bi_bag.pkl'
bags[5] = 'description_tri_bag.pkl'

for filename in bags:
    series = pd.read_pickle(filename)
    df[filename[:-4]] = series

modified = df['priority'].replace('--', 6).replace('P1', 1).replace('P2', 2).replace('P3', 3).replace('P4', 4).replace('P5', 5)
df['priority'] = modified

df.to_pickle('all_in_one.pkl')

In [2]:
import pickle
import pandas as pd
f = open('dictionary.pkl', 'rb')
dictionary = pickle.load(f)
df = pd.read_pickle('all_in_one.pkl')
df.set_index('bug_id', inplace=True)
lines = open('large-eclipse.txt').readlines()

#ID   0
#S-U  1
#S-B  2
#S-T  3
#D-U  4
#D-B  5
#D-T  6
#A-U  7
#A-B  8
#A-T  9
#DID  10
#VERSION       11
#COMPONENT     12
#SUB_COMPONENT 13
#TYPE          14
#PRIORITY      15
with open('eclipse_initial_component.txt','w') as out:
    for i in range(0, len(lines), 16):
        ID = int(lines[i].strip().split('=')[1])
        id_line = lines[i].strip()
        did_line = lines[i + 10].strip()
        version_line = lines[i + 11].strip()
        type_line = lines[i + 14].strip()
        if ID in df.index:
            out.write(id_line)
            out.write('\n')
            out.write( 'S-U=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bag'].items())))
            out.write('\n')
            out.write( 'S-B=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bi_bag'].items())))
            out.write('\n')
            out.write( 'S-T=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_tri_bag'].items())))
            out.write('\n')
            out.write( 'D-U=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'description_bag'].items())))
            out.write('\n')
            out.write( 'D-B=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'description_bi_bag'].items())))
            out.write('\n')
            out.write( 'D-T=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'description_tri_bag'].items())))
            out.write('\n')
            out.write( 'A-U=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bag'].items() + df.ix[ID, 'description_bag'].items())))
            out.write('\n')
            out.write( 'A-B=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bag'].items() + df.ix[ID, 'description_bi_bag'].items())))
            out.write('\n')
            out.write( 'A-T=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bag'].items() + df.ix[ID, 'description_tri_bag'].items())))
            out.write('\n')
            out.write( did_line)
            out.write('\n')
            out.write( version_line)
            out.write('\n')
            out.write( "COMPONENT=" + str(df.ix[ID, 'product_id']))
            out.write('\n')
            out.write( "SUB-COMPONENT=" + str(df.ix[ID, 'component_id']))
            out.write('\n')
            out.write( type_line)
            out.write('\n')
            out.write( "PRIORITY=" + str(df.ix[ID, 'priority']))
            out.write('\n')

In [ ]:
#5. initial priority

In [1]:
from collections import defaultdict
from collections import Counter
import MySQLdb
import pandas as pd
import numpy as np
import sys
reload(sys)
sys.setdefaultencoding('utf8')

conn = MySQLdb.connect(
                     host='localhost', port=3306, user='***', passwd='***', db = 'eclipse2011',
                     charset='utf8')

bug_info = pd.read_sql("SELECT bug_id, short_desc, product_id, component_id, priority FROM bugs WHERE bug_id >= 214049 and bug_id <= 259814;", conn)
bug_info.set_index('bug_id', inplace=True)
activities = pd.read_sql('SELECT * FROM bugs_activity where bug_id >= 214049 and bug_id <= 259814 and fieldid = 13 ' , conn)

product_df = pd.read_sql('SELECT  name,id FROM products ', conn)
product_df.set_index('name', inplace=True)
product_mapping = product_df['id'].to_dict()

component_df = pd.read_sql('SELECT  name,id FROM components ', conn)
component_df.set_index('name', inplace=True)
component_mapping = component_df['id'].to_dict()

activities.sort('bug_when', ascending=False, inplace=True)

after = bug_info.copy()

for rownum in range(0, activities.shape[0]):
    row = activities.iloc[rownum]
    bug_id = row['bug_id']
    field = row['fieldid']
    removed = row['removed']
    try:
        if field == 13:
            after.ix[bug_id, 'priority'] = removed
    except Exception:
        pass

df = after.copy().reset_index()

from nltk.tokenize import WordPunctTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
tokenizer = WordPunctTokenizer()
stemmer = PorterStemmer()

def get_bigrams(string):
    try:
        l = filter(lambda x: x.isalpha(), tokenizer.tokenize(string.lower()))
        bigrams = []
        for i in range(1, len(l)):
            bigrams.append(l[i-1] + ' ' + l[i])
        return bigrams
    except Exception:
        return []

def get_trigrams(string):
    try:
        l = filter(lambda x: x.isalpha(), tokenizer.tokenize(string.lower()))
        bigrams = []
        for i in range(2, len(l)):
            bigrams.append(l[i-2] + ' ' + l[i-1] + ' ' + l[i])
        return bigrams
    except Exception:
        return []

short_desc_bag = df['short_desc'].apply(lambda string: Counter(filter(lambda x: x not in stopwords and x.isalpha(), map(lambda x: stemmer.stem(x), tokenizer.tokenize(string.lower())))))
short_desc_bag.to_pickle('short_desc_bag.pkl')
short_desc_bag = None

short_desc_bi_bag = df['short_desc'].apply(lambda string: Counter(get_bigrams(string)))
short_desc_bi_bag.to_pickle('short_desc_bi_bag.pkl')
short_desc_bi_bag = None

short_desc_tri_bag = df['short_desc'].apply(lambda string: Counter(get_trigrams(string)))
short_desc_tri_bag.to_pickle('short_desc_tri_bag.pkl')
short_desc_tri_bag = None

df.to_pickle('bug_info.pkl')
df = None

description = pd.read_sql('select longdescs.bug_id, longdescs.thetext from longdescs, (select bug_id, MIN(comment_id) as comment_id from longdescs group by bug_id) as t where longdescs.bug_id = t.bug_id and longdescs.comment_id = t.comment_id and longdescs.bug_id >= 214049 and longdescs.bug_id <= 259814', conn)

description_bag = description['thetext'].apply(lambda string: Counter(filter(lambda x: x not in stopwords and x.isalpha(), map(lambda x: stemmer.stem(x), tokenizer.tokenize(string.lower())))))
description_bag.to_pickle('description_bag.pkl')
description_bag = None

description_bi_bag = description['thetext'].apply(lambda string: Counter(get_bigrams(string)))
description_bi_bag.to_pickle('description_bi_bag.pkl')
description_bi_bag = None

description_tri_bag = description['thetext'].apply(lambda string: Counter(get_trigrams(string)))
description_tri_bag.to_pickle('description_tri_bag.pkl')
description_tri_bag = None

description.to_pickle('description.pkl')
description = None

bags = [None] * 6
bags[0] = 'short_desc_bag.pkl'
bags[1] = 'short_desc_bi_bag.pkl'
bags[2] = 'short_desc_tri_bag.pkl'
bags[3] = 'description_bag.pkl'
bags[4] = 'description_bi_bag.pkl'
bags[5] = 'description_tri_bag.pkl'

all_words = set()
for filename in bags:
    series = pd.read_pickle(filename)
    for data in series:
        for word in data:
            all_words.add(word)
            
dictionary = dict((v,k) for k, v in enumerate(all_words))

all_words = None

import pickle
f = open('dictionary.pkl', 'wb')
pickle.dump(dictionary, f)
f.close()

import pickle
import pandas as pd
f = open('dictionary.pkl', 'rb')
dictionary = pickle.load(f)

df = pd.read_pickle('bug_info.pkl')
bags = [None] * 6
bags[0] = 'short_desc_bag.pkl'
bags[1] = 'short_desc_bi_bag.pkl'
bags[2] = 'short_desc_tri_bag.pkl'
bags[3] = 'description_bag.pkl'
bags[4] = 'description_bi_bag.pkl'
bags[5] = 'description_tri_bag.pkl'

for filename in bags:
    series = pd.read_pickle(filename)
    df[filename[:-4]] = series

modified = df['priority'].replace('--', 6).replace('P1', 1).replace('P2', 2).replace('P3', 3).replace('P4', 4).replace('P5', 5)
df['priority'] = modified

df.to_pickle('all_in_one.pkl')

In [2]:
import pickle
import pandas as pd
f = open('dictionary.pkl', 'rb')
dictionary = pickle.load(f)
df = pd.read_pickle('all_in_one.pkl')
df.set_index('bug_id', inplace=True)
lines = open('large-eclipse.txt').readlines()

#ID   0
#S-U  1
#S-B  2
#S-T  3
#D-U  4
#D-B  5
#D-T  6
#A-U  7
#A-B  8
#A-T  9
#DID  10
#VERSION       11
#COMPONENT     12
#SUB_COMPONENT 13
#TYPE          14
#PRIORITY      15
with open('eclipse_initial_priority.txt','w') as out:
    for i in range(0, len(lines), 16):
        ID = int(lines[i].strip().split('=')[1])
        id_line = lines[i].strip()
        did_line = lines[i + 10].strip()
        version_line = lines[i + 11].strip()
        type_line = lines[i + 14].strip()
        if ID in df.index:
            out.write(id_line)
            out.write('\n')
            out.write( 'S-U=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bag'].items())))
            out.write('\n')
            out.write( 'S-B=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bi_bag'].items())))
            out.write('\n')
            out.write( 'S-T=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_tri_bag'].items())))
            out.write('\n')
            out.write( 'D-U=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'description_bag'].items())))
            out.write('\n')
            out.write( 'D-B=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'description_bi_bag'].items())))
            out.write('\n')
            out.write( 'D-T=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'description_tri_bag'].items())))
            out.write('\n')
            out.write( 'A-U=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bag'].items() + df.ix[ID, 'description_bag'].items())))
            out.write('\n')
            out.write( 'A-B=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bag'].items() + df.ix[ID, 'description_bi_bag'].items())))
            out.write('\n')
            out.write( 'A-T=' + ','.join(map(lambda pair: str(dictionary[pair[0]]) + ':' + str(pair[1]), df.ix[ID, 'short_desc_bag'].items() + df.ix[ID, 'description_tri_bag'].items())))
            out.write('\n')
            out.write( did_line)
            out.write('\n')
            out.write( version_line)
            out.write('\n')
            out.write( "COMPONENT=" + str(df.ix[ID, 'product_id']))
            out.write('\n')
            out.write( "SUB-COMPONENT=" + str(df.ix[ID, 'component_id']))
            out.write('\n')
            out.write( type_line)
            out.write('\n')
            out.write( "PRIORITY=" + str(df.ix[ID, 'priority']))
            out.write('\n')